In [2]:
import sys
sys.path.append("/home/yuahe/projects/BERTMap")
from bertmap.map import OntoMapping
from bertmap.onto import Ontology
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [4]:
data_base = "/home/yuahe/projects/BERTMap/largebio_data/onto_labels/"
exp = OntoMapping("fma", "nci", data_base + "fma2nci.small.labels.tsv", data_base + "nci2fma.small.labels.tsv", None, task_suffix="small", name="test_exp")

In [5]:
src_embeds = torch.load(data_base + "fma2nci.small.mean.pt")
tgt_embeds = torch.load(data_base + "nci2fma.small.mean.pt")

In [6]:
src_batch_generator = Ontology.iri_lexicon_batch_generator(exp.src_onto_lexicon_path, batch_size=100)
tgt_batch_generator = Ontology.iri_lexicon_batch_generator(exp.tgt_onto_lexicon_path, batch_size=1000)

for src_batch in src_batch_generator:
    src_batch_embeds = src_embeds[src_batch.index]
    max_scores_list = []
    argmax_scores_list = []
    batch_mapping = pd.DataFrame(columns=["Entity1", "Entity2", "Value"])
    j = 0
    for tgt_batch in tgt_batch_generator:
        tgt_batch_embeds = tgt_embeds[tgt_batch.index]
        sim_scores = cosine_similarity(src_batch_embeds, tgt_batch_embeds)
        max_scores, argmax_scores = sim_scores.max(axis=1), sim_scores.argmax(axis=1) + j * len(tgt_batch)
        max_scores_list.append(torch.tensor(max_scores))
        argmax_scores_list.append(torch.tensor(argmax_scores))
        # display(torch.tensor(max_scores), torch.tensor(argmax_scores))
        j += 1
    batch_max_scores_all = torch.stack(max_scores_list)
    batch_argmax_scores_all = torch.stack(argmax_scores_list)
    batch_max_scores, batch_argmax_scores_inds = batch_max_scores_all.max(axis=0)  # value, indices
    batch_argmax_scores = torch.gather(batch_argmax_scores_all, dim=0, index=batch_argmax_scores_inds.unsqueeze(0).repeat(j, 1))[0]
    batch_mapping["Entity1"] = src_batch["Entity-IRI"]
    batch_mapping["Entity2"] = list(exp.tgt_onto_lexicon["Entity-IRI"].iloc[list(batch_argmax_scores.numpy())])
    batch_mapping["Value"] = batch_max_scores
    display(batch_mapping)
    break


,Entity1,Entity2,Value
0,fma:Concept_name,nci:Gene_Structure,0.922171
1,fma:Part_of_relationship_value,nci:Cell_Signaling_Process,0.928095
2,fma:Physical_anatomical_entity,nci:Experimental_Organism_Anatomical_Concepts,0.960175
3,fma:Non_physical_anatomical_entity_template,nci:Experimental_Organism_Anatomical_Concepts,0.952875
4,fma:Anatomical_structure,nci:Anatomical_Sciences,0.964866
...,...,...,...
95,fma:Histiocyte,nci:Histiocyte,0.984697
96,fma:Alveolar_sac,nci:Alveolar_Sac,0.989405
97,fma:Alveolus,nci:Alveolus,0.979720
98,fma:Alveus_of_hippocampus,nci:Uvea,0.973392
